<span style="font-size:1.1em;">Colab'a Google drive'ı entegre ediyoruz. Kullanılacak olan veriseti Google Drive'da bulunmaktadır</span>

In [134]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


<span style="font-size:1.1em;">Google Drive'ımızın root pathi</span> ```gdrive/My Drive``` <span style="font-size:1.1em;">oluyor. Proje için gerekli verisetini **mbti** adında bir klasör oluşturup içerisine yüklüyoruz. İlgili verisetinin pathi</span> ```gdrive/My Drive/mbti/all_users.csv``` <span style="font-size:1.1em;">oluyor.</span>

In [0]:
PREPROCESSED_DATASET_WITH_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_with_stemming.csv"
PREPROCESSED_DATASET_WITHOUT_STEMMER = "gdrive/My Drive/mbti/preprocessed_dataset_no_stemming.csv"
PREPROCESSED_DATASET_ZEMBEREK = "gdrive/My Drive/mbti/preprocessed_dataset_zemberek"
TRIMMED_DATASET = "gdrive/My Drive/mbti/trimmed_dataset.csv"
RAW_DATASET = "gdrive/My Drive/mbti/all_users_v2.csv"

<span style="font-size:1.1em;">Hangi veriseti kullanılarak işlem yapılacaksa yukardaki pathlerden biri seçilir ve parametre olarak verilir.</span>

In [0]:
import pandas as pd 
df = pd.read_csv(PREPROCESSED_DATASET_WITH_STEMMER, sep = ';', header = 0)

In [137]:
df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,0,19991991,ekşi itiraf dö dolaş gel kendi çat problem ger...,ENTJ,analysts,E,N,T,J
1,1,19991991,selda bağca ses dinleyebilmek büyük mutluluk s...,ENTJ,analysts,E,N,T,J
2,2,19991991,eski sevgili mutlu olma isteye in hiçbir ayak ...,ENTJ,analysts,E,N,T,J
3,3,19991991,veda eder a not bırakmak farklı acı mutluluk...,ENTJ,analysts,E,N,T,J
4,4,19991991,ingil aksa ara bayıldık konuşa insa sevimli ge...,ENTJ,analysts,E,N,T,J
...,...,...,...,...,...,...,...,...,...
524835,643907,zaimoglu,zla ibrahimoviç türki katil,ESFJ,sentinels,E,S,F,J
524836,643908,zaimoglu,tarih büyük yalancı şik operasyo aziz yıldır a...,ESFJ,sentinels,E,S,F,J
524837,643909,zaimoglu,akp chp koalisyo hayali koalisyo,ESFJ,sentinels,E,S,F,J
524838,643910,zaimoglu,trabzonlu insa hamsi diye fenerbahçe taraftar ...,ESFJ,sentinels,E,S,F,J


Her tipten eşit sayıda entry alınabilmesi adına öncelikle bir sınıftaki en az olan entry sayısı bulunur

In [138]:
min_entry = df.groupby('typeClass', as_index = False).count().min().entry

min_entry


54913

Eşit sayıda entry olacak şekilde yeni bir dataframe olusturulur

In [139]:
analysts_df = df[df['typeClass'] == 'analysts']
analysts_df = analysts_df.iloc[0 : min_entry]

analysts_df.shape[0]


54913

In [140]:
explorers_df = df[df['typeClass'] == 'explorers']
explorers_df = explorers_df.iloc[0 : min_entry]

explorers_df.shape[0]

54913

In [141]:
sentinels_df = df[df['typeClass'] == 'sentinels']
sentinels_df = sentinels_df.iloc[0 : min_entry]

sentinels_df.shape[0]

54913

In [142]:
diplomats_df = df[df['typeClass'] == 'diplomats']

diplomats_df = diplomats_df.iloc[0 : min_entry]

diplomats_df.shape[0]

54913

Yeni bir dataframe oluşturulur.

In [143]:
equal_entries_df = pd.concat([diplomats_df, sentinels_df, explorers_df, analysts_df]).reset_index(drop=True)
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,240593,1903 24,gtü den evr gün ne soruşturma ıee gtü den açık...,INFJ,diplomats,I,N,F,J
1,240594,1903 24,ekşi sözlük speakingi club istanbul hazır,INFJ,diplomats,I,N,F,J
2,240595,1903 24,personalities com sene sür yapmış infj advo...,INFJ,diplomats,I,N,F,J
3,240596,1903 24,müzik dinlemek hara uyuşturuç aşık dertli eser...,INFJ,diplomats,I,N,F,J
4,240599,1903 24,zengi insa kurabilecek cümle al kar sık arar,INFJ,diplomats,I,N,F,J
...,...,...,...,...,...,...,...,...,...
219647,69519,gulcehatun,ceset gör sözlük yazar okuduk görmedi düşündür...,INTJ,analysts,I,N,T,J
219648,69520,gulcehatun,nişa dük tektaş bala istemeye istemeye mümk is...,INTJ,analysts,I,N,T,J
219649,69522,gulcehatun,yürüye merdive duran uyarıl kampanyas istanbul...,INTJ,analysts,I,N,T,J
219650,69523,gulcehatun,zaman baş atla in mesel ağladık gözyaş ke silm...,INTJ,analysts,I,N,T,J


Oluşturulan dataframe shuffle edilir.

In [0]:
equal_entries_df = equal_entries_df.sample(frac=1).reset_index(drop=True)

In [145]:
equal_entries_df

,Unnamed: 0,user,entry,type,typeClass,I/E,S/N,T/F,J/P
0,584729,melancohly,ohannesburger alsancak dolaşır meka görüp bir ...,ESTJ,sentinels,E,S,T,J
1,480740,damatferid,how met your mother dan akıl kalan sezo bö...,ESTP,explorers,E,S,T,P
2,523160,sn3x,medeniyet gösterges küçük detay insa birbir gü...,ISFP,explorers,I,S,F,P
3,575515,incir agaci golgesinde,ev ödev kaldırılmas saçma olay gelecek seçim a...,ISTJ,sentinels,I,S,T,J
4,36391,cok gezen adam,çocuk okuna dergi milliyet çocuk,INTJ,analysts,I,N,T,J
...,...,...,...,...,...,...,...,...,...
219647,295265,hakkican21,buket a ın kanal d den kovulmas adet you nh...,INFJ,diplomats,I,N,F,J
219648,502827,kanuni sozluk celebi,kayınpeder damadı aşık seviyos git konuş,ISFP,explorers,I,S,F,P
219649,465202,bardan adam,fifa söylememiş denk gelmedi oy vicenza marc...,ESTP,explorers,E,S,T,P
219650,564606,fyodor fyodorovic,aykut kocama futbol anlaya futbol teorik matem...,ISFJ,sentinels,I,S,F,J


<span style="font-size:1.1em;">TF-IDF özellik vektörünün çıkartılmasında kullanılacak değişken aşağıda belirlenmiş olan parametrelerle oluşturulur.</span>

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True, norm='l2', encoding='utf-8', ngram_range=(1, 2), max_features=100, analyzer = 'word', token_pattern=r'\w{1,}')

**Feature Extraction** 

Burada tf-idf vektörlerinin çıkartılmasında kullanılacak olan vocabulary oluşturulur.

In [0]:
import numpy as np

equal_entries_df['entry'] = equal_entries_df['entry'].apply(lambda x: np.str_(x)) # ValueError: np.nan is an invalid document seklinde bir hata verdigi icin bunu asmak adina yapildi.

In [148]:
tfidf_vectorizer.fit(equal_entries_df['entry'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=100,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=True, token_pattern='\\w{1,}', tokenizer=None,
                use_idf=True, vocabulary=None)

Veriseti train ve test olmak üzere ikiye ayrılır. Test %20 ve train %80'ini oluşturacak şekilde tüm veriseti bölünür. random_state parametresi ile tekrardan bölündüğünde bir öncekiyle aynı train ve test veri setlerinin oluşturulması sağlanır.

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['typeClass'], random_state = 42, test_size = 0.20, shuffle = True)

y_actual = y_test.values

In [151]:
X_train.shape[0]

175721

In [152]:
X_test.shape[0]

43931

Train ve test datasetlerinden tf-idf vektörleri çıkartılır


In [0]:
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [154]:
X_train_tfidf

<175721x100 sparse matrix of type '<class 'numpy.float64'>'
	with 698544 stored elements in Compressed Sparse Row format>

In [155]:
X_test_tfidf

<43931x100 sparse matrix of type '<class 'numpy.float64'>'
	with 175643 stored elements in Compressed Sparse Row format>

SVM kullanılarak tahmin edilir.

In [156]:
from sklearn import svm
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_predicted = SVM.predict(X_test_tfidf)

In [158]:
y_predicted

array(['explorers', 'explorers', 'sentinels', ..., 'analysts',
       'explorers', 'analysts'], dtype=object)

<span style="font-size:1.1em">Yapılacak tahminlerle ilgili istatistiksel verileri tutmak için</span> ```predictions_result```<span style="font-size:1.1em"> adında bir değişken oluşturulur.</span>

<span style="font-size:1.1em">Bu değişkenin yapısı aşağıdaki gibidir.</span>

```json
{
    "predicted": {
        "analysts":  { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "diplomats": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "explorers": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
        "sentinels": { "actual": {"analysts": 0, "diplomats": 0, "explorers": 0, "sentinels": 0} }
    }
}
```

* <span style="font-size:1.1em;">Yapılan tahminlerle ilgili verilere ulaşabilmek için</span>

    ```predictions_results['predicted']```


* <span style="font-size:1.1em;">Yapılan tahminin analyst ise:</span>

    ```predictions_results['predicted']['analysts']``` 


* <span style="font-size:1.1em;">Yapılan analyst tahmininin gerçek değerlerine erişmek için:</span>     

    ```predictions_results['predicted']['analysts']['actual']```  


* <span style="font-size:1.1em;">Test verisi, model tarafından analysts olarak tahmin edilmiştir ve bu verinin gerçek değeri de analysts'tir.</span>

    ```predictions_results['predicted']['analysts']['actual']['analysts']``` 

In [0]:
prediction_results = {'predicted': {}}  ## prediction_result['analysts'] means prediction is 'analysts'

prediction_results['predicted']['analysts']  = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['diplomats'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['explorers'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}
prediction_results['predicted']['sentinels'] = {'actual': {'analysts': 0, 'diplomats': 0, 'explorers': 0, 'sentinels': 0}}

## prediction_result['analysts']['diplomats'] means prediction is analysts but actual value is diplomats

```prediction_results```<span style="font-size:1.1em"> içerisinde tutulan sayaçların değerleri arttırılır.</span>

In [0]:
for i in range(len(y_predicted)):
  predicted_value = y_predicted[i]
  actual_value = y_actual[i]
  prediction_results['predicted'][predicted_value]['actual'][actual_value] += 1

<span style="font-size:1.1em">JSON formatına çevrilir </span>```dict``` <span style="font-size:1.1em">tipi. Bu sayede daha okunaklı bir şekilde print edilmiş olur. </span>

In [161]:
import json

print(json.dumps(prediction_results, indent = 2))

{
  "predicted": {
    "analysts": {
      "actual": {
        "analysts": 2569,
        "diplomats": 2047,
        "explorers": 2130,
        "sentinels": 2222
      }
    },
    "diplomats": {
      "actual": {
        "analysts": 896,
        "diplomats": 1558,
        "explorers": 885,
        "sentinels": 840
      }
    },
    "explorers": {
      "actual": {
        "analysts": 5080,
        "diplomats": 5100,
        "explorers": 5539,
        "sentinels": 4991
      }
    },
    "sentinels": {
      "actual": {
        "analysts": 2430,
        "diplomats": 2294,
        "explorers": 2402,
        "sentinels": 2948
      }
    }
  }
}


<span style="font-size:1.1em;">İlgili field extract edilir</span> ```dict``` <span style="font-size:1.1em;">yapısından.</span>

In [0]:
results = prediction_results['predicted']

<span style="font-size:1.1em;">Başarı oranı hesaplanır</span>

In [163]:
accuracy = (results['analysts']['actual']['analysts'] + results['diplomats']['actual']['diplomats'] + results['explorers']['actual']['explorers'] + results['sentinels']['actual']['sentinels']) / len(y_predicted)
accuracy

0.28713209351027746

<span style="font-size:1.1em;">**E/I** boyutu tahmin edilir</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['I/E'], random_state = 42, shuffle = True)  ## Geri kalanlar S, T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [165]:
y_predicted

array(['E', 'E', 'E', ..., 'E', 'E', 'E'], dtype=object)

In [166]:
predicted = {}
predicted['I'] = {'actual': {'I': 0, 'E': 0}}
predicted['E'] = {'actual': {'I': 0, 'E': 0}}
predicted

{'E': {'actual': {'E': 0, 'I': 0}}, 'I': {'actual': {'E': 0, 'I': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1


In [168]:
predicted

{'E': {'actual': {'E': 26228, 'I': 22878}},
 'I': {'actual': {'E': 2649, 'I': 3158}}}

In [0]:
accuracy = (predicted['E']['actual']['E'] + predicted['I']['actual']['I']) / len(y_predicted)

In [170]:
accuracy

0.5351373991586692

<span style="font-size:1.1em">**S/N** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['S/N'], random_state = 42, shuffle = True)  ## Geri kalan boyutlar: T, J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [172]:
predicted['N'] = {'actual': {'N': 0, 'S': 0}}
predicted['S'] = {'actual': {'N': 0, 'S': 0}}

predicted

{'E': {'actual': {'E': 26228, 'I': 22878}},
 'I': {'actual': {'E': 2649, 'I': 3158}},
 'N': {'actual': {'N': 0, 'S': 0}},
 'S': {'actual': {'N': 0, 'S': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [174]:
predicted

{'E': {'actual': {'E': 26228, 'I': 22878}},
 'I': {'actual': {'E': 2649, 'I': 3158}},
 'N': {'actual': {'N': 6789, 'S': 5564}},
 'S': {'actual': {'N': 20708, 'S': 21852}}}

In [0]:
accuracy = (predicted['N']['actual']['N'] + predicted['S']['actual']['S']) / len(y_predicted)

In [176]:
accuracy

0.5215704842204942

<span style="font-size:1.1em">**T/F** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['T/F'], random_state = 42, shuffle = True)  ##  J

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [178]:
predicted['T'] = {'actual': {'T': 0, 'F': 0}}
predicted['F'] = {'actual': {'T': 0, 'F': 0}}

predicted

{'E': {'actual': {'E': 26228, 'I': 22878}},
 'F': {'actual': {'F': 0, 'T': 0}},
 'I': {'actual': {'E': 2649, 'I': 3158}},
 'N': {'actual': {'N': 6789, 'S': 5564}},
 'S': {'actual': {'N': 20708, 'S': 21852}},
 'T': {'actual': {'F': 0, 'T': 0}}}

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [180]:
predicted

{'E': {'actual': {'E': 26228, 'I': 22878}},
 'F': {'actual': {'F': 3328, 'T': 2348}},
 'I': {'actual': {'E': 2649, 'I': 3158}},
 'N': {'actual': {'N': 6789, 'S': 5564}},
 'S': {'actual': {'N': 20708, 'S': 21852}},
 'T': {'actual': {'F': 23266, 'T': 25971}}}

In [0]:
accuracy = (predicted['F']['actual']['F'] + predicted['T']['actual']['T']) / len(y_predicted)

In [182]:
accuracy

0.5335530748638756

<span style="font-size:1.1em">**J/P** boyutu tahmin edilir.</span>

In [0]:
X_train, X_test, y_train, y_test = train_test_split(equal_entries_df['entry'], equal_entries_df['J/P'], random_state = 42, shuffle = True) 

X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(X_train_tfidf,y_train)

y_actual = y_test.values

y_predicted = SVM.predict(X_test_tfidf)

In [0]:
predicted['J'] = {'actual': {'J': 0, 'P': 0}}
predicted['P'] = {'actual': {'J': 0, 'P': 0}}

predicted

In [0]:
for i in range(len(y_predicted)):
  predicted[y_predicted[i]]['actual'][y_actual[i]] += 1

In [0]:
predicted

In [0]:
accuracy = (predicted['P']['actual']['P'] + predicted['J']['actual']['J']) / len(y_predicted)

In [0]:
accuracy

In [0]:
prediction_results['predicted'].update(predicted) 

prediction_results